In [1]:
import tensorflow as tf
from tensorflow.contrib import predictor

In [185]:
export_dir = r"gs://named_entity_recognition/ml_engine/export/exporter/1547053937/"
predict_fn = predictor.from_saved_model(export_dir)


INFO:tensorflow:Restoring parameters from gs://named_entity_recognition/ml_engine/export/exporter/1547053937/variables/variables


In [186]:
from string import ascii_lowercase
MAPPING = {a:index for index,a in enumerate(ascii_lowercase + ascii_lowercase.upper())}
class PadList(list):
    """ The super padding list used for padding data
    """
    def inner_pad(self, pad_length, pad_value=0):
        """Do inner padding of the list
        
        Paramters:
            padd_length -- How long should the list be
            padd_value -- What value should be used for the padding
        
        Return:
            self -- the list 
        """
        nbr_pad = pad_length - len(self)
        if nbr_pad>0:
            self = self + [pad_value] * nbr_pad
        else:
            self=self[:pad_length]
        return self
    
    
    def outer_pad(self,padded_list_length,pad_length,pad_value=0):
        """
        Out padding of a list e.g append a list to a list. 
        Args:
            padded_list_length -- how long should the appended list be
            pad_lenght -- how long should the list be e.g how much should we append
            padd_value -- What should the appended list have as values
        """
        nbr_pad = pad_length-len(self)
        if nbr_pad > 0:
            for _ in range(nbr_pad):
                self.append([pad_value] * padded_list_length)
        else:
            self = self[:pad_length]
        return self

In [187]:
def get_chars(text=None):
    #text = element["text"]
    UNKONWN=52
    PADD_VALUE_CHAR=UNKONWN+1
    MAX_WORD_LEN=15
    MAX_SENTENCE_LEN=25
    mapped=PadList()
    for word in text.split(" "):
        tmp_mapped=PadList()
        for char in word:
            try:
                tmp_mapped.append(MAPPING[char.strip()])
            except:
                tmp_mapped.append(UNKONWN)
        tmp_mapped=tmp_mapped.inner_pad(MAX_WORD_LEN,PADD_VALUE_CHAR)
        mapped.append(tmp_mapped)
    mapped = mapped.outer_pad(padded_list_length=MAX_WORD_LEN,pad_length=MAX_SENTENCE_LEN,pad_value=PADD_VALUE_CHAR)
    return mapped

In [188]:
def word_lengths(text=None):
    MAX_WORD_LEN=15
    MAX_SENTENCE_LEN=25
    word_lengths  = PadList([len(word) if len(word)<MAX_WORD_LEN else MAX_WORD_LEN for word in text.split()])
    return word_lengths.inner_pad(MAX_SENTENCE_LEN,0)

In [189]:
predictor_fn = tf.contrib.predictor.from_saved_model(
                export_dir=export_dir,
        )

INFO:tensorflow:Restoring parameters from gs://named_entity_recognition/ml_engine/export/exporter/1547053937/variables/variables


In [190]:
predictor_fn.feed_tensors

{u'chars': <tf.Tensor 'ParseExample/ParseExample:0' shape=(?, 25, 15) dtype=int64>,
 u'chars_in_word': <tf.Tensor 'ParseExample/ParseExample:1' shape=(?, 25) dtype=int64>,
 u'sentence_length': <tf.Tensor 'ParseExample/ParseExample:2' shape=(?,) dtype=int64>,
 u'text': <tf.Tensor 'ParseExample/ParseExample:3' shape=(?,) dtype=string>}

In [197]:
text=" Mr Niklas will move to London"
data = {
        'text': [text],
        "chars" : [get_chars(text)],
        'sentence_length':[len((text.strip().split(" ")))],
        'chars_in_word': [word_lengths(text)],
        }

In [198]:
out = predictor_fn(data)
out

{u'pred_ids': array([[0, 3, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0]], dtype=int32),
 u'tags': array([['O', 'B-PER', 'O', 'O', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O',
         'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O',
         'O']], dtype=object)}

In [199]:
predictions = predict_fn(data)
print(predictions)

{u'pred_ids': array([[0, 3, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0]], dtype=int32), u'tags': array([['O', 'B-PER', 'O', 'O', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O',
        'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O',
        'O']], dtype=object)}


# Summary
It seams like it has a hard time with short sentences. Longer sentences are much better!
Lets see what happens if i let the model run longer. Set up a propper long training run tomorrow! 